## Тест: Множественная проверка гипотез
### Вопрос 1.
Выберите задачи, в которых обязательно нужно применять поправку на множественную проверку гипотез
- Всё

### Вопрос 2.
Классификатор C4.5 и три его модификации: с оптимизацией гиперпараметра m, гиперпараметра cf и с одновременной оптимизацией обоих гиперпараметров. Эти четыре классификатора сравнивались на 14 наборах данных. На каждом датасете был посчитан AUC каждого классификатора. Данные записаны в файле:
AUCs.txt

Используя критерий знаковых рангов, проведите попарное сравнение каждого классификатора с каждым. Выберите два классификатора, различие между которыми наиболее статистически значимо. 

In [1]:
import pandas as pd

data = pd.read_csv('AUCs.txt', sep='\t')
data.head()

,Unnamed: 0,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


In [2]:
from scipy import stats
columns = ['C4.5', 'C4.5+m', 'C4.5+cf', 'C4.5+m+cf']
corr_data = []

for i, lhs_column in enumerate(columns):
    for j, rhs_column in enumerate(columns):
        if i >= j:
            continue
        
        corr, p = stats.wilcoxon(data[lhs_column], data[rhs_column])
        corr_data.append([lhs_column, rhs_column, corr, p])
#stats.wilcoxon(weight_data.After, weight_data.Before)

In [3]:
corr_data

[['C4.5', 'C4.5+m', 6.5, 0.01075713311978963],
 ['C4.5', 'C4.5+cf', 43.0, 0.86126233009534803],
 ['C4.5', 'C4.5+m+cf', 11.0, 0.015906444101703374],
 ['C4.5+m', 'C4.5+cf', 17.0, 0.046332729793395394],
 ['C4.5+m', 'C4.5+m+cf', 22.0, 0.32782567584464062],
 ['C4.5+cf', 'C4.5+m+cf', 10.0, 0.022909099354356588]]

In [4]:
print 'Answer: [\'C4.5\', \'C4.5+m\']'

Answer: ['C4.5', 'C4.5+m']


### Вопрос 3.
Сколько статистически значимых на уровне 0.05 различий мы обнаружили?

In [5]:
for i, item in enumerate(corr_data):
    if item[3] < 0.05:
        print item

['C4.5', 'C4.5+m', 6.5, 0.01075713311978963]
['C4.5', 'C4.5+m+cf', 11.0, 0.015906444101703374]
['C4.5+m', 'C4.5+cf', 17.0, 0.046332729793395394]
['C4.5+cf', 'C4.5+m+cf', 10.0, 0.022909099354356588]


### Вопрос 4.
Судя по данным из предыдущего опроса, настройка какого из параметров классификатора даёт более значимое увеличение качества?

In [6]:
print 'Answer: m'

Answer: m


### Вопрос 5.
Сравнивая 4 классификатора между собой, мы проверили 6 гипотез. Давайте сделаем поправку на множественную проверку. Начнём с метода Холма. Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки этим методом?

In [7]:
from statsmodels.sandbox.stats.multicomp import multipletests 

sales_correlation = pd.DataFrame.from_records(corr_data)
sales_correlation.columns = ['clf_A', 'clf_B', 'corr', 'p']
reject, p_corrected, a1, a2 = multipletests(sales_correlation.p, 
                                            alpha = 0.05, 
                                            method = 'holm')

In [8]:
sales_correlation['p_corrected'] = p_corrected
sales_correlation['reject'] = reject
sales_correlation.reject.value_counts()

False    6
Name: reject, dtype: int64

In [9]:
print 'answer', 0

answer 0


### Вопрос 6.
Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки методом Бенджамини-Хохберга? 

In [10]:
reject, p_corrected, a1, a2 = multipletests(sales_correlation.p, 
                                            alpha = 0.05, 
                                            method = 'fdr_bh') 

In [11]:
sales_correlation['p_corrected'] = p_corrected
sales_correlation['reject'] = reject
sales_correlation.reject.value_counts()

True     3
False    3
Name: reject, dtype: int64

### Вопрос 7.
Насколько корректно, на ваш взгляд, применение метода Бенджамини-Хохберга в этой задаче?
- Некорректно: статистики, соответствующие разным гипотезам, зависимы, так что предположения метода Бенджамини-Хохберга не выполняются

### Вопрос 8.
Мы подозреваем, что в проведённом с C4.5 эксперименте на самом деле классификаторы сильнее отличаются друг от друга, просто нам не удалось это заметить. Что можно сделать, чтобы увеличить вероятность обнаружения различий, если они действительно существуют?
- Взять больше датасетов